# Goals
- To prepare a mini version of the data sources for extracting the affliation of the papers. That is, the origin of the papers' data, which we assume the data is collected from the regions/countries that the authors are from.
- To identify the countries of the data origin for each paper. 

In [20]:
%load_ext autoreload
%autoreload 2

In [21]:
import numpy as np
import pandas as pd

# Get only the location information of each papers
- None of the columns are unique id, including cord_uid. But, we will use this for now because it has the most unique ids. 

In [48]:
# load the data columns and remove some reducdant information
col_to_remove = ['text', 'bibliography']
col_names = pd.read_csv('data/merged_raw_data.csv', nrows=1).columns.tolist()
for c in col_to_remove:
    col_names.remove(c)
col_names = ['cord_uid', 'affiliations', 'location']

# load the data
df = pd.read_csv('data/merged_raw_data.csv', usecols=col_names)

# check
print(df.shape)
df.head(5)

(51078, 3)


,cord_uid,affiliations,location
0,xqhn0vbp,NaN,NaN
1,gi6uaa83,Todd R Disotell (New York University ; 25 Wave...,25 Waverly Place ; 10003 ; New York ; NY ; USA
2,le0ogx1s,NaN,NaN
3,fy4w7xz8,NaN,NaN
4,0qaoam29,NaN,NaN


In [50]:
# save
df.to_csv('data/merged_raw_data_location.csv')

# Load the data and then identify their location
- geograpy
    - geograpy1 or 2 did not work because of the errors in installation: https://github.com/Corollarium/geograpy2 has installation issues. DID NOT USE IT
    - geograpy3 works: https://github.com/jmbielec/geograpy3
- can GeoText as well: https://github.com/elyase/geotext
- look like the geograpy3 is more robust to bad text (although not 100% correct) that can detect more regions, but the precision may be low
- may be we should combine both the use of geograpy and geotext

In [2]:
from geotext import GeoText

In [43]:
# load data
df = pd.read_csv('data/merged_raw_data_location.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df.dropna()
df = df.reset_index(drop=True)

# create fake columns
df['affilation_extraction'] = 0
df['location_extraction'] = 0

# check
print(df.shape)
df.head(3)



(18558, 5)


,cord_uid,affiliations,location,affilation_extraction,location_extraction
0,gi6uaa83,Todd R Disotell (New York University ; 25 Wave...,25 Waverly Place ; 10003 ; New York ; NY ; USA,0,0
1,1wswi7us,Yee Leng Yap ; Xue Wu Zhang ; Antoine Danchin ...,"; ; 75724, Cedex 15 ; Paris ; France",0,0
2,yy96yeu9,David Wang (University of California San Franc...,San Francisco ; California ; United States of ...,0,0


In [54]:
# loop through each address and make classification
for i in range(0, df.shape[0]):
    text = df.iloc[i]['affiliations']
    print(text)
    
    # geograpy3 extraction
    places = geograpy3.get_place_context(text = text)
    print(places.country_cities)
    
    # GeoText extraction
    places = GeoText(text).country_mentions
    print(places)
    dsa

Todd R Disotell (New York University ; 25 Waverly Place ; 10003 ; New York ; NY ; USA)
{'United States': ['New York', 'Todd'], 'Argentina': ['Todd']}
OrderedDict([('US', 3)])


NameError: name 'dsa' is not defined

# Testing

In [3]:
import geograpy3
text_input = "Yin-Jing Tien (National Central University ; 32001 ; Tao-Yuan ; Taiwan) ; Yun-Shien Lee (33305 ; Tao-Yuan ; Taiwan) ; Han-Ming Wu (Tamkang University ; 25137 ; Tamsui ; Taiwan) ; Chun-Houh Chen (Academia Sinica ; 11529 ; Taipei ; Taiwan)"
places = geograpy3.get_place_context(text = text_input)

print(places.countries)
print(places.country_mentions)
print(places.country_cities)
print()

places = GeoText(text_input).country_mentions
print(places)







['Taiwan', 'Tamkang University', 'National Central University', 'Tamsui', 'Chen', 'Academia Sinica', 'Tien', 'Taipei']
[('Taiwan', 4), ('Tien', 1), ('National Central University', 1), ('Tamkang University', 1), ('Tamsui', 1), ('Chen', 1), ('Academia Sinica', 1), ('Taipei', 1)]
{'Taiwan': ['Taipei']}

OrderedDict([('TW', 5), ('US', 1)])
